In [ ]:
# NLP-Based Resume-Job Description Compatibility Checker with Enhanced Features

# Install required packages
!pip install -q gradio sentence-transformers PyPDF2 nltk scikit-learn python-docx

import gradio as gr
import PyPDF2
import nltk
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from io import BytesIO
import io
from docx import Document
from collections import Counter

# Download NLTK data
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

print("✅ All packages installed successfully!")

# ============================================
# 1. LOAD EMBEDDING MODEL
# ============================================
print("Loading Sentence Transformer model...")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
print("✅ Model loaded successfully!")

# ============================================
# 2. TEXT EXTRACTION FUNCTIONS
# ============================================

def extract_text_from_pdf(pdf_file):
    """Extract text from uploaded PDF file"""
    try:
        if isinstance(pdf_file, str):
            with open(pdf_file, 'rb') as f:
                pdf_reader = PyPDF2.PdfReader(f)
                text = ""
                for page in pdf_reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
        else:
            if isinstance(pdf_file, bytes):
                pdf_file = io.BytesIO(pdf_file)
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in pdf_reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text.strip()
    except Exception as e:
        return f"Error extracting PDF: {str(e)}"

def extract_text_from_docx(docx_file):
    """Extract text from uploaded Word (.docx) file"""
    try:
        if isinstance(docx_file, str):
            doc = Document(docx_file)
        else:
            if isinstance(docx_file, bytes):
                docx_file = io.BytesIO(docx_file)
            doc = Document(docx_file)

        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"

        for table in doc.tables:
            for row in table.rows:
                for cell in row.cells:
                    text += cell.text + " "
            text += "\n"

        return text.strip()
    except Exception as e:
        return f"Error extracting Word document: {str(e)}"

def extract_text_from_file(file):
    """Universal function to extract text from PDF or Word files"""
    if file is None:
        return "No file uploaded"

    if isinstance(file, str):
        filename = file
    else:
        filename = file.name if hasattr(file, 'name') else ""

    file_ext = filename.lower().split('.')[-1]

    if file_ext == 'pdf':
        return extract_text_from_pdf(file)
    elif file_ext in ['docx', 'doc']:
        return extract_text_from_docx(file)
    else:
        return f"Unsupported file format: .{file_ext}. Please upload PDF or Word (.docx) files."

def clean_text(text):
    """Clean and normalize text"""
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,;:()\-]', '', text)
    return text.strip()

# ============================================
# 3. NEW FEATURE: BIAS & VAGUE LANGUAGE DETECTOR
# ============================================

def identify_section_at_position(resume_text, position):
    """Identify which section of the resume a position falls into"""
    text_lower = resume_text.lower()

    # Define section patterns with their identifiers
    section_headers = [
        ('experience', r'(work experience|professional experience|experience|employment history|work history)'),
        ('education', r'(education|academic background|qualifications|degree)'),
        ('skills', r'(technical skills|skills|core competencies|expertise|technologies)'),
        ('projects', r'(projects|key projects|notable projects|portfolio)'),
        ('summary', r'(summary|professional summary|profile|objective|about)'),
        ('certifications', r'(certifications|certificates|licenses)'),
        ('achievements', r'(achievements|accomplishments|awards|honors)')
    ]

    # Find all section headers and their positions
    sections = []
    for section_name, pattern in section_headers:
        for match in re.finditer(pattern, text_lower):
            sections.append({
                'name': section_name,
                'start': match.start(),
                'end': match.end()
            })

    # Sort sections by position
    sections.sort(key=lambda x: x['start'])

    # Determine which section the position falls into
    for i, section in enumerate(sections):
        section_start = section['end']
        section_end = sections[i + 1]['start'] if i + 1 < len(sections) else len(resume_text)

        if section_start <= position < section_end:
            return section['name'].title()

    # If not in any identified section
    return "Other"

def detect_vague_language(resume_text):
    """Detect weak, vague, or biased phrases in resume with section identification"""
    vague_patterns = {
        'weak_verbs': {
            'patterns': [
                r'\b(responsible for|duties included|worked on|helped with|assisted in|involved in|participated in)\b',
            ],
            'message': 'Weak action verb - replace with strong action verbs',
            'examples': 'Use: Led, Developed, Achieved, Implemented, Designed instead'
        },
        'vague_quantifiers': {
            'patterns': [
                r'\b(many|several|various|numerous|multiple|some)\b',
            ],
            'message': 'Vague quantifier - add specific numbers',
            'examples': 'Replace "many" with "15+" or "dozens of"'
        },
        'unclear_impact': {
            'patterns': [
                r'\b(improved|enhanced|optimized|increased|reduced)\b(?!\s+by\s+\d)',
            ],
            'message': 'Missing quantifiable impact - add percentages or metrics',
            'examples': 'Add: "by 35%", "by $50K", "from 2hrs to 30min"'
        },
        'passive_voice': {
            'patterns': [
                r'\b(was|were|been)\s+\w+ed\b',
            ],
            'message': 'Passive voice detected - use active voice',
            'examples': 'Change "was developed by me" to "Developed"'
        },
        'filler_words': {
            'patterns': [
                r'\b(very|really|quite|somewhat|fairly|rather)\b',
            ],
            'message': 'Filler word - remove for stronger impact',
            'examples': 'Remove words like "very skilled" → "skilled"'
        },
        'personal_pronouns': {
            'patterns': [
                r'\b(I|my|me|we|our|us)\b',
            ],
            'message': 'Personal pronoun - remove for professional tone',
            'examples': 'Remove "I led" → "Led"'
        }
    }

    issues_found = []
    text_lower = resume_text.lower()

    for category, data in vague_patterns.items():
        for pattern in data['patterns']:
            matches = list(re.finditer(pattern, text_lower, re.IGNORECASE))
            for match in matches:
                start = max(0, match.start() - 50)
                end = min(len(resume_text), match.end() + 50)
                context = resume_text[start:end].strip()

                # Identify which section this issue is in
                section = identify_section_at_position(resume_text, match.start())

                issues_found.append({
                    'category': category.replace('_', ' ').title(),
                    'phrase': match.group(),
                    'context': context,
                    'message': data['message'],
                    'examples': data['examples'],
                    'position': match.start(),
                    'section': section  # NEW: Section identifier
                })

    issues_found.sort(key=lambda x: x['position'])
    return issues_found

# ============================================
# 4. NEW FEATURE: ATS COMPLIANCE SCORING
# ============================================

def check_ats_compliance(resume_text):
    """Check if resume is ATS-friendly"""
    compliance_score = 100
    issues = []
    warnings = []

    # Check 1: Special characters and symbols
    special_chars = re.findall(r'[★☆●○■□▪▫◆◇]', resume_text)
    if special_chars:
        issues.append(f"❌ Special characters found ({len(special_chars)}) - replace with standard text")
        compliance_score -= 15

    # Check 2: Tables detection
    if resume_text.count('|') > 10:
        warnings.append("⚠️ Possible table formatting - ATS may misread columns")
        compliance_score -= 10

    # Check 3: Contact information
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    phone_pattern = r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b'

    has_email = bool(re.search(email_pattern, resume_text))
    has_phone = bool(re.search(phone_pattern, resume_text))

    if not has_email:
        issues.append("❌ Email address not detected")
        compliance_score -= 15
    if not has_phone:
        warnings.append("⚠️ Phone number not clearly detected")
        compliance_score -= 5

    # Check 4: Section headers
    standard_sections = ['experience', 'education', 'skills', 'work', 'professional']
    sections_found = sum(1 for section in standard_sections if section in resume_text.lower())

    if sections_found < 2:
        issues.append("❌ Missing standard section headers (Experience, Education, Skills)")
        compliance_score -= 20

    # Check 5: Date formats
    date_patterns = [
        r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+\d{4}\b',
        r'\b\d{1,2}/\d{4}\b',
        r'\b\d{4}\s*-\s*\d{4}\b'
    ]
    dates_found = sum(1 for pattern in date_patterns if re.search(pattern, resume_text))

    if dates_found == 0:
        warnings.append("⚠️ No clear date formats found - add dates to experience")
        compliance_score -= 10

    # Check 6: Acronyms
    acronyms = re.findall(r'\b[A-Z]{2,}\b', resume_text)
    if len(acronyms) > 10:
        warnings.append(f"⚠️ Many acronyms found ({len(acronyms)}) - consider expanding first use")
        compliance_score -= 5

    # Check 7: Length
    word_count = len(resume_text.split())
    if word_count < 200:
        issues.append("❌ Resume too short (< 200 words)")
        compliance_score -= 15
    elif word_count > 1000:
        warnings.append("⚠️ Resume lengthy (> 1000 words) - consider condensing")
        compliance_score -= 5

    # Check 8: Hyperlinks
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls_found = len(re.findall(url_pattern, resume_text))
    if urls_found > 3:
        warnings.append("⚠️ Multiple URLs found - ATS may not process hyperlinks")
        compliance_score -= 5

    return {
        'score': max(0, compliance_score),
        'issues': issues,
        'warnings': warnings,
        'word_count': word_count,
        'has_email': has_email,
        'has_phone': has_phone,
        'sections_found': sections_found
    }

# ============================================
# 5. NEW FEATURE: RESUME REWRITE SUGGESTIONS
# ============================================

def generate_rewrite_suggestions(resume_text, jd_text, skill_gaps):
    """Generate specific rewrite suggestions based on job description"""
    suggestions = []
    jd_lower = jd_text.lower()

    # Suggestion 1: Add missing skills
    if skill_gaps['missing']:
        top_missing = skill_gaps['missing'][:5]
        suggestions.append({
            'priority': 'HIGH',
            'section': 'Skills',
            'issue': f"Missing {len(skill_gaps['missing'])} key skills from job description",
            'suggestion': f"Add these skills if you have them: {', '.join(top_missing)}",
            'example': f"Technical Skills: Python, SQL, {', '.join(top_missing[:3])}, ..."
        })

    # Suggestion 2: Quantify achievements
    vague_achievements = re.findall(
        r'(improved|increased|reduced|enhanced|optimized|developed|created|led|managed)',
        resume_text.lower()
    )
    if len(vague_achievements) > 3:
        suggestions.append({
            'priority': 'HIGH',
            'section': 'Experience',
            'issue': 'Multiple achievements lack quantifiable metrics',
            'suggestion': 'Add specific numbers, percentages, or timeframes to show impact',
            'example': 'Before: "Improved system performance"\nAfter: "Improved system performance by 45%, reducing load time from 3s to 1.5s"'
        })

    # Suggestion 3: Match job title keywords
    jd_titles = re.findall(r'(engineer|developer|analyst|manager|scientist|specialist)', jd_lower)
    resume_lower = resume_text.lower()

    if jd_titles and not any(title in resume_lower for title in jd_titles):
        suggestions.append({
            'priority': 'MEDIUM',
            'section': 'Professional Summary',
            'issue': 'Resume doesn\'t emphasize role titles from job description',
            'suggestion': f'Include relevant titles like: {", ".join(set(jd_titles[:3]))}',
            'example': f'Add: "Experienced {jd_titles[0].title()} with expertise in..."'
        })

    # Suggestion 4: Add action verbs
    weak_verbs = len(re.findall(r'\b(responsible for|worked on|helped)\b', resume_text.lower()))
    if weak_verbs > 2:
        suggestions.append({
            'priority': 'MEDIUM',
            'section': 'Experience',
            'issue': f'Found {weak_verbs} weak action verbs',
            'suggestion': 'Replace with strong action verbs that show leadership and impact',
            'example': 'Replace "Responsible for managing" with "Led" or "Directed"\nReplace "Worked on projects" with "Developed" or "Architected"'
        })

    # Suggestion 5: Education requirements
    if 'bachelor' in jd_lower or 'degree' in jd_lower:
        if not re.search(r'(bachelor|master|phd|degree)', resume_text.lower()):
            suggestions.append({
                'priority': 'HIGH',
                'section': 'Education',
                'issue': 'Education section missing or not prominent',
                'suggestion': 'Clearly list your degree(s) as job requires specific education',
                'example': 'Bachelor of Science in Computer Science, XYZ University, 2020'
            })

    # Suggestion 6: Add projects
    if 'project' in jd_lower and 'project' not in resume_text.lower():
        suggestions.append({
            'priority': 'MEDIUM',
            'section': 'Projects',
            'issue': 'No projects section found but job mentions project experience',
            'suggestion': 'Add 2-3 relevant projects that demonstrate required skills',
            'example': 'Projects:\n• NLP Chatbot - Built using Python, TensorFlow, achieving 90% accuracy\n• Data Pipeline - Processed 1M+ records using SQL and Python'
        })

    # Suggestion 7: Years of experience
    years_required = re.findall(r'(\d+)\+?\s*years?', jd_lower)
    if years_required:
        suggestions.append({
            'priority': 'LOW',
            'section': 'Professional Summary',
            'issue': f'Job requires {years_required[0]} years experience',
            'suggestion': 'Highlight your years of experience early in resume',
            'example': f'Professional Summary: Software Engineer with {years_required[0]}+ years...'
        })

    return suggestions

# ============================================
# 6. SECTION EXTRACTION
# ============================================

def extract_resume_sections(resume_text):
    """Extract different sections from resume"""
    sections = {
        'experience': '',
        'education': '',
        'skills': '',
        'projects': '',
        'other': resume_text
    }

    patterns = {
        'experience': r'(work experience|professional experience|experience|employment history|work history)',
        'education': r'(education|academic background|qualifications)',
        'skills': r'(technical skills|skills|core competencies|expertise|technologies)',
        'projects': r'(projects|key projects|notable projects|portfolio)'
    }

    text_lower = resume_text.lower()
    section_positions = []

    for section_name, pattern in patterns.items():
        matches = list(re.finditer(pattern, text_lower))
        for match in matches:
            section_positions.append((match.start(), section_name))

    section_positions.sort()

    for i, (start_pos, section_name) in enumerate(section_positions):
        if i < len(section_positions) - 1:
            end_pos = section_positions[i + 1][0]
            sections[section_name] = resume_text[start_pos:end_pos]
        else:
            sections[section_name] = resume_text[start_pos:]

    return sections

def analyze_section_strength(section_text, jd_text):
    """Analyze how well a resume section matches the job description"""
    if not section_text or len(section_text.strip()) < 20:
        return 0.0

    section_embedding = model.encode([section_text])
    jd_embedding = model.encode([jd_text])
    similarity = cosine_similarity(section_embedding, jd_embedding)[0][0]

    return round(similarity * 100, 2)

# ============================================
# 7. KEYWORD EXTRACTION
# ============================================

def extract_keywords(text, top_n=20):
    """Extract important keywords from text"""
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    keywords = [
        word for word in tokens
        if word.isalnum() and word not in stop_words and len(word) > 2
    ]

    keyword_freq = Counter(keywords)
    return keyword_freq.most_common(top_n)

def find_matching_keywords(resume_text, jd_text):
    """Find common keywords between resume and job description"""
    resume_keywords = set([kw[0] for kw in extract_keywords(resume_text, top_n=50)])
    jd_keywords = set([kw[0] for kw in extract_keywords(jd_text, top_n=50)])
    matching = resume_keywords.intersection(jd_keywords)
    return sorted(list(matching))

# ============================================
# 8. SKILL GAP ANALYSIS
# ============================================

def extract_technical_skills(text):
    """Extract technical skills from text"""
    skill_patterns = [
        r'\b(python|java|javascript|typescript|c\+\+|c#|ruby|go|rust|swift|kotlin|php|r|matlab|scala)\b',
        r'\b(react|angular|vue|node\.?js|express|django|flask|spring|tensorflow|pytorch|keras|scikit-learn|pandas|numpy)\b',
        r'\b(sql|mysql|postgresql|mongodb|redis|elasticsearch|oracle|sqlite|dynamodb)\b',
        r'\b(aws|azure|gcp|docker|kubernetes|jenkins|terraform|ansible|git|github|gitlab)\b',
        r'\b(machine learning|deep learning|nlp|computer vision|data science|neural networks|transformers|bert|gpt|llm)\b',
        r'\b(jupyter|colab|tableau|power bi|excel|jira|confluence|slack)\b',
        r'\b(html|css|rest api|graphql|websocket|ajax)\b',
        r'\b(agile|scrum|ci/cd|microservices|api|backend|frontend|full stack)\b'
    ]

    text_lower = text.lower()
    skills = set()

    for pattern in skill_patterns:
        matches = re.findall(pattern, text_lower, re.IGNORECASE)
        skills.update(matches)

    return skills

def analyze_skill_gaps(resume_text, jd_text):
    """Identify missing skills from job description"""
    resume_skills = extract_technical_skills(resume_text)
    jd_skills = extract_technical_skills(jd_text)

    matching_skills = resume_skills.intersection(jd_skills)
    missing_skills = jd_skills - resume_skills
    extra_skills = resume_skills - jd_skills

    return {
        'matching': sorted(list(matching_skills)),
        'missing': sorted(list(missing_skills)),
        'extra': sorted(list(extra_skills)),
        'match_percentage': round(len(matching_skills) / len(jd_skills) * 100, 2) if jd_skills else 0
    }

# ============================================
# 9. COMPATIBILITY SCORING
# ============================================

def calculate_compatibility(resume_text, jd_text):
    """Calculate semantic similarity between resume and JD"""
    resume_embedding = model.encode([resume_text])
    jd_embedding = model.encode([jd_text])
    similarity = cosine_similarity(resume_embedding, jd_embedding)[0][0]
    score = round(similarity * 100, 2)
    return score

# ============================================
# 10. MAIN ANALYSIS FUNCTION
# ============================================

def analyze_resume(file, job_description):
    """Main function to analyze resume compatibility"""

    if file is None:
        return "❌ Please upload a resume (PDF or Word)", "", "", "", "", "", "", "", ""

    if not job_description or len(job_description.strip()) < 50:
        return "❌ Please enter a valid job description", "", "", "", "", "", "", "", ""

    try:
        resume_text = extract_text_from_file(file)

        if resume_text.startswith("Error") or resume_text.startswith("Unsupported"):
            return f"❌ {resume_text}", "", "", "", "", "", "", "", ""

        if not resume_text or len(resume_text) < 50:
            return "❌ Could not extract sufficient text from file", "", "", "", "", "", "", "", ""

        clean_resume = clean_text(resume_text)
        clean_jd = clean_text(job_description)

        # Original analyses
        overall_score = calculate_compatibility(clean_resume, clean_jd)
        sections = extract_resume_sections(resume_text)
        section_scores = {}
        for section_name, section_text in sections.items():
            if section_text and len(section_text.strip()) > 20:
                score = analyze_section_strength(section_text, clean_jd)
                section_scores[section_name] = score

        matching_keywords = find_matching_keywords(clean_resume, clean_jd)
        skill_gaps = analyze_skill_gaps(resume_text, job_description)

        # New features
        vague_language_issues = detect_vague_language(resume_text)
        ats_compliance = check_ats_compliance(resume_text)
        rewrite_suggestions = generate_rewrite_suggestions(resume_text, job_description, skill_gaps)

        # Format results
        score_text = f"## 🎯 Overall Compatibility Score: **{overall_score}%**\n\n"
        if overall_score >= 80:
            score_text += "✅ **Excellent Match!**"
        elif overall_score >= 60:
            score_text += "✔️ **Good Match!**"
        elif overall_score >= 40:
            score_text += "⚠️ **Moderate Match.**"
        else:
            score_text += "❌ **Low Match.**"

        # Section Analysis
        section_analysis = "## 📊 Section Analysis\n\n"
        if section_scores:
            sorted_sections = sorted(section_scores.items(), key=lambda x: x[1], reverse=True)
            section_analysis += "| Section | Score | Status |\n|---------|-------|--------|\n"
            for section_name, score in sorted_sections:
                emoji = "🟢" if score >= 60 else "🟡" if score >= 40 else "🔴"
                status = "Strong" if score >= 60 else "Moderate" if score >= 40 else "Weak"
                section_analysis += f"| {section_name.title()} | {score}% | {emoji} {status} |\n"

        # Skill Gaps
        skill_gap_text = f"## 🎯 Skill Gap Analysis\n\n**Match Rate:** {skill_gaps['match_percentage']}%\n\n"
        if skill_gaps['matching']:
            skill_gap_text += f"### ✅ Matching ({len(skill_gaps['matching'])})\n"
            skill_gap_text += ", ".join([f"**{s}**" for s in skill_gaps['matching'][:15]]) + "\n\n"
        if skill_gaps['missing']:
            skill_gap_text += f"### ❌ Missing ({len(skill_gaps['missing'])})\n"
            skill_gap_text += ", ".join([f"**{s}**" for s in skill_gaps['missing'][:20]]) + "\n\n"

        # Keywords
        keywords_text = "### 🔑 Matching Keywords:\n\n"
        if matching_keywords:
            keywords_text += ", ".join([f"**{kw}**" for kw in matching_keywords[:20]])
        else:
            keywords_text += "⚠️ No significant matches found"

        # Vague Language
        vague_language_text = "## 🔍 Vague Language Detector\n\n"
        if vague_language_issues:
            vague_language_text += f"**Found {len(vague_language_issues)} issues**\n\n"

            # Group by category AND section
            issues_by_cat = {}
            for issue in vague_language_issues[:20]:  # Show up to 20 issues
                cat = issue['category']
                if cat not in issues_by_cat:
                    issues_by_cat[cat] = []
                issues_by_cat[cat].append(issue)

            for category, issues in issues_by_cat.items():
                vague_language_text += f"### ⚠️ {category} ({len(issues)} found)\n"
                vague_language_text += f"**Issue:** {issues[0]['message']}\n"
                vague_language_text += f"**Fix:** {issues[0]['examples']}\n\n"

                # Show examples with section information
                vague_language_text += "**Examples in your resume:**\n"
                for issue in issues[:5]:  # Show up to 5 examples per category
                    vague_language_text += f"- **[{issue['section']}]** *\"{issue['phrase']}\"* in: ...{issue['context']}...\n"
                vague_language_text += "\n"
        else:
            vague_language_text += "✅ No major issues detected!\n"

        # ATS Compliance
        ats_text = f"## 🤖 ATS Compliance: **{ats_compliance['score']}%**\n\n"
        if ats_compliance['score'] >= 80:
            ats_text += "✅ **Excellent** - ATS-friendly\n\n"
        elif ats_compliance['score'] >= 60:
            ats_text += "✔️ **Good** - Minor fixes needed\n\n"
        else:
            ats_text += "⚠️ **Needs Improvement**\n\n"

        if ats_compliance['issues']:
            ats_text += "### ❌ Issues:\n"
            for issue in ats_compliance['issues']:
                ats_text += f"- {issue}\n"
        if ats_compliance['warnings']:
            ats_text += "\n### ⚠️ Warnings:\n"
            for warning in ats_compliance['warnings']:
                ats_text += f"- {warning}\n"

        # Rewrite Suggestions
        rewrite_text = "## ✍️ Rewrite Suggestions\n\n"
        if rewrite_suggestions:
            high = [s for s in rewrite_suggestions if s['priority'] == 'HIGH']
            medium = [s for s in rewrite_suggestions if s['priority'] == 'MEDIUM']

            if high:
                rewrite_text += "### 🔴 HIGH PRIORITY\n\n"
                for i, sug in enumerate(high, 1):
                    rewrite_text += f"**{i}. {sug['section']}:** {sug['suggestion']}\n"
                    rewrite_text += f"```\n{sug['example']}\n```\n\n"

            if medium:
                rewrite_text += "### 🟡 MEDIUM PRIORITY\n\n"
                for i, sug in enumerate(medium, 1):
                    rewrite_text += f"**{i}. {sug['section']}:** {sug['suggestion']}\n\n"
        else:
            rewrite_text += "✅ Looks good!\n"

        # Preview
        preview_text = "### 📄 Preview:\n\n" + clean_resume[:2000]
        if len(clean_resume) > 2000:
            preview_text += "..."

        return (score_text, section_analysis, skill_gap_text, keywords_text,
                vague_language_text, ats_text, rewrite_text, preview_text, clean_resume)

    except Exception as e:
        import traceback
        return f"❌ Error: {str(e)}\n\n{traceback.format_exc()}", "", "", "", "", "", "", "", ""

# ============================================
# 11. GRADIO INTERFACE
# ============================================

SAMPLE_JD = """
"""

custom_css = """
.gradio-container {
    font-family: 'Arial', sans-serif;
}
.output-markdown h2 {
    color: #2c3e50;
    border-bottom: 2px solid #3498db;
    padding-bottom: 10px;
}
"""

with gr.Blocks(css=custom_css, title="Enhanced Resume Analyzer") as demo:

    gr.Markdown("""
# Enhanced Resume Analyzer

Evaluates resume–job fit beyond keywords, analyzes language quality, estimates ATS readiness, and suggests targeted improvements.

### 🧭 How to Use
1. Upload your resume (PDF or Word)
2. Paste the full job description
3. Click **Analyze**
4. Review results tab-by-tab, starting with **Overview**
""")


    with gr.Row():
        with gr.Column(scale=1):


            file_input = gr.File(
                label="Upload Resume (PDF/Word)",
                file_types=[".pdf", ".docx"],
                type="filepath"
            )

            jd_input = gr.Textbox(
                label="Job Description",
                placeholder="Paste job description here...",
                lines=10,
                value=SAMPLE_JD
            )

            with gr.Row():
                analyze_btn = gr.Button("🔍 Analyze", variant="primary", size="lg")
                clear_btn = gr.Button("🔄 Clear", size="lg")

        with gr.Column(scale=1):
            gr.Markdown("### 📊 Results")

            with gr.Tabs():
                with gr.Tab("📈 Overview"):
                    score_output = gr.Markdown()
                    section_output = gr.Markdown()

                with gr.Tab("🎯 Skills"):
                    skill_gap_output = gr.Markdown()
                    keywords_output = gr.Markdown()

                with gr.Tab("🔍 Language"):
                    vague_language_output = gr.Markdown()

                with gr.Tab("🤖 ATS"):
                    ats_output = gr.Markdown()

                with gr.Tab("✍️ Suggestions"):
                    rewrite_output = gr.Markdown()

                with gr.Tab("📄 Preview"):
                    preview_output = gr.Markdown()

            full_text_output = gr.Textbox(visible=False)

    # Connect buttons
    analyze_btn.click(
        fn=analyze_resume,
        inputs=[file_input, jd_input],
        outputs=[score_output, section_output, skill_gap_output, keywords_output,
                vague_language_output, ats_output, rewrite_output, preview_output, full_text_output]
    )

    def clear_all():
      return None, SAMPLE_JD, "", "", "", "", "", "", "", ""

# This should be at the same indentation level as the function above
    clear_btn.click(
        fn=clear_all,
        inputs=[],
        outputs=[file_input, jd_input, score_output, section_output, skill_gap_output,
                keywords_output, vague_language_output, ats_output, rewrite_output,
                preview_output, full_text_output]
    )



    gr.Markdown("""
---
**Built as an applied NLP system for resume–job alignment analysis.**
Heuristic-based; not a hiring, ranking, or evaluation tool.
""")


# Launch
print("\n" + "="*50)
print("🚀 Launching Enhanced Resume Analyzer...")
print("="*50 + "\n")

demo.launch(share=True, debug=True)

✅ All packages installed successfully!
Loading Sentence Transformer model...
✅ Model loaded successfully!


/tmp/ipython-input-1679794164.py:708: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, title="Enhanced Resume Analyzer") as demo:



🚀 Launching Enhanced Resume Analyzer...

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d1f136ea92ce454705.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2202, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1924, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1879, in validate_outputs
    rai